# Balanced BSTs

## Agenda

1. Motivation
2. "Balanced" binary trees
3. AVL trees
4. Red-black trees
5. Summary

## 1. Motivation

In [2]:
class BSTree:
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right
            
    def __init__(self):
        self.size = 0
        self.root = None
    
    def add(self, val):
        assert val not in self
        def add_rec(node):
            if not node:
                return BSTree.Node(val)
            elif val < node.val:
                node.left = add_rec(node.left)
                return node
            else:
                node.right = add_rec(node.right)
                return node
        self.root = add_rec(self.root)
        self.size += 1

    def __contains__(self, val):
        def contains_rec(node):
            if not node:
                return False
            elif val == node.val:
                return True
            elif val < node.val:
                return contains_rec(node.left)
            else:
                return contains_rec(node.right)
        return contains_rec(self.root)        
                        
    def pprint(self, width=64):
        """Attempts to pretty-print this tree's contents."""
        height = self.height()
        nodes  = [(self.root, 0)]
        prev_level = 0
        repr_str = ''
        while nodes:
            n,level = nodes.pop(0)
            if prev_level != level:
                prev_level = level
                repr_str += '\n'
            if not n:
                if level < height-1:
                    nodes.extend([(None, level+1), (None, level+1)])
                repr_str += '{val:^{width}}'.format(val='-', width=width//2**level)
            elif n:
                if n.left or level < height-1:
                    nodes.append((n.left, level+1))
                if n.right or level < height-1:
                    nodes.append((n.right, level+1))
                repr_str += '{val:^{width}}'.format(val=n.val, width=width//2**level)
        print(repr_str)
    
    def height(self):
        """Returns the height of the longest branch of the tree."""
        def height_rec(t):
            if not t:
                return 0
            else:
                return max(1+height_rec(t.left), 1+height_rec(t.right))
        return height_rec(self.root)

t = BSTree()
for x in range(6):
    t.add(x)
t.pprint()

                               0                                
               -                               1                
       -               -               -               2        
   -       -       -       -       -       -       -       3    
 -   -   -   -   -   -   -   -   -   -   -   -   -   -   -   4  
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 5 


I.e., if a binary search tree is not balanced, it may degrade to a linked list!

## 2. "Balanced" binary trees

There are different criteria we can use to decide whether a binary tree is "balanced", e.g., 

- the *number* of nodes on either side of a given node
- the *height* of the subtrees on either side of a given node
- the *density* of nodes on either side of a given node
- the *shape* of the subtrees on either side of a given node (perhaps we only want to permit full or complete trees).

What are the pros and cons of some of these criteria?

### Balance factor

*def.* in a height-balanced tree, the *balance factor* of a node is the height of its right subtree minus the height of its left subtree.

![](../images/16-avl-bfegs.jpg)

In [3]:
# Update `add` so as to maintain the balance factor of each node in the tree.

# Key question: how does adding a node affect the balance factor of its 
# ancestors? Remember that a node is always added as a leaf!

class BSTree(BSTree):
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right
            self.bf = 0
            
    def add(self, val):
        assert val not in self
        def add_rec(node):
            if not node:
                return (1, BSTree.Node(val))
            elif val < node.val:
                dh, node.left = add_rec(node.left)
                if dh:
                    if node.bf <= 0:
                        node.bf -= 1
                        dh = 1
                    else:
                        node.bf -= dh
                        dh = 0
            else:
                dh, node.right = add_rec(node.right)
                if dh:
                    if node.bf >= 0:
                        node.bf += 1
                        dh = 1
                    else:
                        node.bf += dh
                        dh = 0
            return (dh, node)
        _, self.root = add_rec(self.root)
        self.size += 1

    def pprint(self, width=64):
        """Attempts to pretty-print this tree's contents."""
        height = self.height()
        nodes  = [(self.root, 0)]
        prev_level = 0
        repr_str = ''
        while nodes:
            n,level = nodes.pop(0)
            if prev_level != level:
                prev_level = level
                repr_str += '\n'
            if not n:
                if level < height-1:
                    nodes.extend([(None, level+1), (None, level+1)])
                repr_str += f'{"-":^{width//2**level}}'
            elif n:
                if n.left or level < height-1:
                    nodes.append((n.left, level+1))
                if n.right or level < height-1:
                    nodes.append((n.right, level+1))
                repr_str += f'{f"{n.val}[{n.bf}]":^{width//2**level}}'
        print(repr_str)

In [19]:
import random
vals = list(range(10))
random.shuffle(vals)

t = BSTree()
for x in vals:
    t.add(x)
t.pprint()

                              4[0]                              
              0[3]                           9[-3]              
       -             3[-2]            5[2]             -        
   -       -     2[-1]     -       -      7[0]     -       -    
 -   -   -   -  1[0] -   -   -   -   -  6[0]8[0] -   -   -   -  


## 3. AVL (Adelson-Velsky and Landis) trees

*def.* an AVL tree is a height-balanced binary search tree where the balance factor is either -1, 0, or 1 for every node.

An AVL tree is *self-balancing*. I.e., whenever the AVL property may be violated by an insertion/deletion, it is immediately "fixed" by the implementation.

How do we go about fixing an out-of-balance BST?

### 3.1. Essential operation: "rotation"

When the balance factor of a binary search tree node is < -1 (left-heavy) or > 1 (right heavy)

What we need is a procedure that:
1. does not change the in-order traversal of the tree
2. does not change the keys or values of the nodes
3. does not change the binary search tree property
4. does change the balance factors of the nodes
5. does change the height of the nodes

![](../images/16-avl-rotations.jpg)

In [ ]:
class AVLTree(BSTree):
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right

        def rotate_right(self):
            pass
            
    def add(self, val):
        assert val not in self
        def add_rec(node):
            if not node:
                return AVLTree.Node(val)
            elif val < node.val:
                node.left = add_rec(node.left)
                return node
            else:
                node.right = add_rec(node.right)
                return node
        self.root = add_rec(self.root)
        self.size += 1

In [ ]:
t = AVLTree()
for x in range(6, 0, -1):
    t.add(x)
t.pprint()

In [ ]:
t.root.rotate_right()
t.pprint()

In [ ]:
t.root.rotate_right()
t.pprint()

In [ ]:
t.root.left.rotate_right()
t.pprint()

In [ ]:
class AVLTree(BSTree):
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right
            
        def rotate_right(self):
            p = self
            c = self.left
            p.val, c.val = c.val, p.val
            p.left, p.right, c.left, c.right = c.left, c, c.right, p.right
        
        @staticmethod
        def height(n):
            if not n:
                return 0
            else:
                return max(1+AVLTree.Node.height(n.left), 1+AVLTree.Node.height(n.right))
                                    
    def add(self, val):
        assert val not in self
        def add_rec(node):
            if not node:
                return AVLTree.Node(val)
            elif val < node.val:
                node.left = add_rec(node.left)
            else:
                node.right = add_rec(node.right)
            # detect and fix imbalance in `node` before returning
            return node
        self.root = add_rec(self.root)
        self.size += 1

In [ ]:
val = 50
t = AVLTree()

In [ ]:
# (evaluate multiple times with ctrl-enter)
t.add(val)
val -= 1
t.pprint()

In [ ]:
val = 0
t = AVLTree()

In [ ]:
# (evaluate multiple times with ctrl-enter)
t.add(val)
val += 1
t.pprint()

### 3.2. "Out-of-balance" scenarios & rotation recipes

![](../images/16-avl-recipes.jpg)

In [ ]:
# "left-left" scenario
t = BSTree()
for x in [3, 2, 1]:
    t.add(x)
t.pprint()

In [ ]:
# "left-right" scenario
t = BSTree()
for x in [3, 1, 2]:
    t.add(x)
t.pprint()

In [ ]:
# "right-right" scenario
t = BSTree()
for x in [1, 2, 3]:
    t.add(x)
t.pprint()

In [ ]:
# "right-left" scenario
t = BSTree()
for x in [1, 3, 2]:
    t.add(x)
t.pprint()

### 3.3. AVL tree insertion

In [ ]:
class AVLTree(BSTree):
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right

        def rotate_right(self):
            p = self
            c = self.left
            p.val, c.val = c.val, p.val
            p.left, p.right, c.left, c.right = c.left, c, c.right, p.right
            
        def rotate_left(self):
            pass
        
        @staticmethod
        def height(n):
            if not n:
                return 0
            else:
                return max(1+AVLTree.Node.height(n.left), 1+AVLTree.Node.height(n.right))
    
    @staticmethod
    def rebalance(node):
        if AVLTree.Node.height(node.left) > AVLTree.Node.height(node.right):
            if AVLTree.Node.height(node.left.left) >= AVLTree.Node.height(node.left.right):
                # left-left
                print('left-left imbalance detected')
                # fix?
            else:
                # left-right
                print('left-right imbalance detected')
                # fix?
        else:
            # right branch imbalance tests needed
            pass
            
    def add(self, val):
        assert val not in self
        def add_rec(node):
            if not node:
                return AVLTree.Node(val)
            elif val < node.val:
                node.left = add_rec(node.left)
            else:
                node.right = add_rec(node.right)
            if False: # detect imbalance
                AVLTree.rebalance(node)
            return node
        self.root = add_rec(self.root)
        self.size += 1

In [ ]:
t = AVLTree()
for x in [10, 5, 1]:
    t.add(x)
t.pprint()

In [ ]:
# broken!
t = AVLTree()
for x in [10, 5, 1, 2, 3]:
    t.add(x)
t.pprint()

### 3.4. AVL tree removal

In [ ]:
class AVLTree(BSTree):
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right

        def rotate_right(self):
            p = self
            c = self.left
            p.val, c.val = c.val, p.val
            p.left, p.right, c.left, c.right = c.left, c, c.right, p.right
        
        def rotate_left(self):
            pass
            
        @staticmethod
        def height(n):
            if not n:
                return 0
            else:
                return max(1+AVLTree.Node.height(n.left), 1+AVLTree.Node.height(n.right))

    @staticmethod
    def rebalance(node):
        if AVLTree.Node.height(node.left) > AVLTree.Node.height(node.right):
            if AVLTree.Node.height(node.left.left) >= AVLTree.Node.height(node.left.right):
                # left-left
                print('left-left imbalance detected')
                node.rotate_right()
            else:
                # left-right
                print('left-right imbalance detected')
                node.left.rotate_left()
                node.rotate_right()
        else:
            pass
            
    def add(self, val):
        assert val not in self
        def add_rec(node):
            if not node:
                return AVLTree.Node(val)
            elif val < node.val:
                node.left = add_rec(node.left)
            else:
                node.right = add_rec(node.right)
            if abs(AVLTree.Node.height(node.left) - AVLTree.Node.height(node.right)) >= 2:
                AVLTree.rebalance(node)
            return node
        self.root = add_rec(self.root)
        self.size += 1
        
    def __delitem__(self, val):
        assert val in self
        def delitem_rec(node):
            if val < node.val:
                node.left = delitem_rec(node.left)
            elif val > node.val:
                node.right = delitem_rec(node.right)
            else:
                if not node.left and not node.right:
                    return None
                elif node.left and not node.right:
                    return node.left
                elif node.right and not node.left:
                    return node.right
                else:
                    to_del = node.left
                    if not to_del.right:
                        node.left = to_del.left
                    else:
                        par = to_del
                        to_del = par.right
                        while to_del.right:
                            par = par.right
                            to_del = to_del.right
                        par.right = to_del.left
                    node.val = to_del.val

            # detect and fix imbalance
            return node
                        
        self.root = delitem_rec(self.root)
        self.size -= 1

In [ ]:
t = AVLTree()
for x in [10, 5, 15, 2]:
    t.add(x)
t.pprint()

In [ ]:
del t[15]
t.pprint()

In [ ]:
t = AVLTree()
for x in range(31, 0, -1):
    t.add(x)
t.pprint()

In [ ]:
del t[15]
del t[14]
t.pprint()

In [ ]:
del t[16]
t.pprint()

### 3.5. AVL tree analysis

Central to the runtime complexity of AVL tree operations is the height of the tree $h$ in terms of the number of nodes $n$.

![](../images/16-avl-runtime1.jpg)
![](../images/16-avl-runtime2.jpg)

## 4. Red-black trees

## 5. Summary